In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import preprocessing
from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split


In [6]:
df = pd.read_csv('../data/BitcoinHeistData.csv')
df = df.sample(frac=1)
df.head()


,address,year,day,length,weight,count,looped,neighbors,income,label
1074188,19Zh4MZM8nCJF1zQ5LfSTKPps2EFtNdhsn,2013,313,2,1.000000,1,0,1,767793620.0,white
1094888,1G3szpqiRvpzdhKapWDMc53nhrWkBa6vut,2013,334,4,0.750000,2,0,2,53594429.0,white
1107598,1GGFKznMN24CatVz7DX8uszb3UUA3jdWEA,2013,346,144,0.016264,1302,1301,2,497560000.0,white
592066,17Q5HHbMRNdw8KUt1VNz8DGjk6M9G4RiDq,2012,196,138,0.125826,828,0,1,49949999.0,white
1760257,174HHuUjoMNvcSPj5yYJsAVoqcnnXBYuJe,2015,269,144,0.000870,1772,0,2,671756636.0,white


In [3]:
df.isnull().sum()


address      0
year         0
day          0
length       0
weight       0
count        0
looped       0
neighbors    0
income       0
label        0
dtype: int64

In [7]:
def split(df):
    # df = df.drop(['address'], axis=1)
    le = preprocessing.LabelEncoder()
    le.fit(df.address)
    X = df.drop('label', axis=1)
    y = df['label']
    X_train, X_, y_train, y_ = train_test_split(X, y, test_size=0.3)
    X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.5)
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = split(df)

ValueError: could not convert string to float: '141tHBcNqHggmaVrorc6EjfbrCwuywFiPo'

In [5]:
def dt():
    for d in [4, 8, 10, 15, 20]: #15 : 0.9879
        clf = DecisionTreeClassifier(criterion='gini', max_depth=d)
        clf.fit(X_train, y_train)
        print(f'gini, max-depth:{d}, score:{clf.score(X_test, y_test)}')

    for d in [4, 8, 10, 15, 20]:
        clf = DecisionTreeClassifier(criterion='entropy', max_depth=d)
        clf.fit(X_train, y_train)
        print(f'entropy, max-depth:{d}, score:{clf.score(X_test, y_test)}')


dt()

gini, max-depth:4, score:0.98558873612873
gini, max-depth:8, score:0.9862241574382007
gini, max-depth:10, score:0.9864961543296648
gini, max-depth:15, score:0.987915566679238
gini, max-depth:20, score:0.9863292990937247
entropy, max-depth:4, score:0.98558873612873
entropy, max-depth:8, score:0.9858561616438669
entropy, max-depth:10, score:0.9871567182089348
entropy, max-depth:15, score:0.9878698529159666
entropy, max-depth:20, score:0.9860070170626621
